In [5]:
import pandas as pd
import os
import numpy as np
import json
import sys
import time
from pathlib import Path
import subprocess

In [2]:
def getSortResults(output_dir, clu_version):
    # load results from phy for run logging and creation of the table for C_Waves

    cluLabel = np.load(os.path.join(output_dir, 'spike_clusters.npy'))
    spkTemplate = np.load(os.path.join(output_dir,'spike_templates.npy'))
    cluLabel = np.squeeze(cluLabel)
    spkTemplate = np.squeeze(spkTemplate)

    unqLabel, labelCounts = np.unique(cluLabel, return_counts = True)
    nTot = cluLabel.shape[0]
    nLabel = unqLabel.shape[0]
    maxLabel = np.max(unqLabel)

    templates = np.load(os.path.join(output_dir, 'templates.npy'))
    channel_map = np.load(os.path.join(output_dir, 'channel_map.npy'))
    channel_map = np.squeeze(channel_map)
    
    # read in inverse of whitening matrix
    w_inv = np.load((os.path.join(output_dir, 'whitening_mat_inv.npy')))
    nTemplate = templates.shape[0]
    
    # initialize peak_channels array
    peak_channels = np.zeros([nLabel,],'uint32')
    
   
    # After manual splits or merges, some labels will have spikes found with
    # different templats.
    # for each label in the list unqLabel, get the most common template
    # For that template (nt x nchan), multiply the the transpose (nchan x nt) by inverse of 
    # the whitening matrix (nchan x nchan); get max and min along tthe time axis (1)
    # to find the peak channel
    for i in np.arange(0,nLabel):
        curr_spkTemplate = spkTemplate[np.where(cluLabel==unqLabel[i])]
        template_mode = np.argmax(np.bincount(curr_spkTemplate))
        currT = templates[template_mode,:].T
        curr_unwh = np.matmul(w_inv, currT)
        currdiff = np.max(curr_unwh,1) - np.min(curr_unwh,1)
        peak_channels[i] = channel_map[np.argmax(currdiff)]

    clus_Table = np.zeros((maxLabel+1, 2), dtype='uint32')
    clus_Table[unqLabel, 0] = labelCounts
    clus_Table[unqLabel, 1] = peak_channels

    if clu_version == 0:
        np.save(os.path.join(output_dir, 'clus_Table.npy'), clus_Table)
    else:
        clu_Name = 'clus_Table_' + repr(clu_version) + '.npy'
        np.save(os.path.join(output_dir, clu_Name), clus_Table)
 
    return nTemplate, nTot

In [3]:
output_dir = r'D:\NPX_data\DATA\05242024-NTQP4_g3\05242024-NTQP4_g3\kilosort_output'

getSortResults(output_dir, 0)

(151, 745940)

In [ ]:
temp = np.load(clus_lbl_path)
temp = temp.astype('uint32')


np.save(os.path.join(spikeSort_path, 'spike_clusters_converted'), temp)

In [6]:
tools_path = r'D:\NPX_data' # path to the folder holding all the "tools": CatGT, TPrime, etc
input_path = r'D:\NPX_data\DATA\05242024-NTQP4_g3' # path where the recording file is: this needs to be edited
output_path = r'D:\NPX_data\CatGT_OUT' # path to spit out the output of CatGT

base_path = Path('D:/NPX_data/DATA/05242024-NTQP4_g3') 
run_name = '05242024-NTQP4_g3'
spikeSort_path = base_path / run_name / 'kilosort_output'

In [23]:
bin_path = os.path.join(input_path,run_name,'05242024-NTQP4_g3_tcat.imec0.ap.bin')
clus_table_path = os.path.join(spikeSort_path,'clus_Table.npy')
clus_time_path = os.path.join(spikeSort_path,'spike_times.npy')
clus_lbl_path = os.path.join(spikeSort_path,'spike_clusters_converted.npy')
dest_path = os.path.join(spikeSort_path,'C_waves')


In [25]:
# This runs CWaves
command = f'runit.bat \
    -spikeglx_bin={bin_path} \
    -clus_table_npy={clus_table_path} \
    -clus_time_npy={clus_time_path} \
    -clus_lbl_npy={clus_lbl_path} \
    -dest={dest_path} \
    -samples_per_spike=82 \
    -pre_samples=41 \
    -num_spikes=500 \
    -snr_radius=8 \
    -snr_radius_um=140' 
print(command)
print(os.system(f"cd /d {os.path.join(tools_path, 'C_Waves-win')} & {command}"))
print(f'{run_name} done')


runit.bat     -spikeglx_bin=D:\NPX_data\DATA\05242024-NTQP4_g3\05242024-NTQP4_g3\05242024-NTQP4_g3_tcat.imec0.ap.bin     -clus_table_npy=D:\NPX_data\DATA\05242024-NTQP4_g3\05242024-NTQP4_g3\kilosort_output\clus_Table.npy     -clus_time_npy=D:\NPX_data\DATA\05242024-NTQP4_g3\05242024-NTQP4_g3\kilosort_output\spike_times.npy     -clus_lbl_npy=D:\NPX_data\DATA\05242024-NTQP4_g3\05242024-NTQP4_g3\kilosort_output\spike_clusters_converted.npy     -dest=D:\NPX_data\DATA\05242024-NTQP4_g3\05242024-NTQP4_g3\kilosort_output\C_waves     -samples_per_spike=82     -pre_samples=41     -num_spikes=500     -snr_radius=8     -snr_radius_um=140
0
05242024-NTQP4_g3 done
